In [1]:
import os
import numpy as np
import tensorflow as tf
import tensorflow.contrib.rnn as rnn
import tensorflow.contrib.layers as layers
from text_loader import TextLoader 

C:\Python3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
num_layers  = 3
hidden_size = 512
batch_size  = 1 # 1글자
# max_length  = 1 # 1로 하는것으로~

loader = TextLoader("sentences_for_word2vec.txt")
vocab_size = len(loader.vocab)

In [3]:
X = tf.placeholder(tf.float32, [None, None, 100])
# x_one_hot = tf.one_hot(X, vocab_size)

cells = [rnn.BasicLSTMCell(hidden_size) for _ in range(num_layers)]
cells = rnn.MultiRNNCell(cells, state_is_tuple=True)

initial_state = cells.zero_state(batch_size, tf.float32)
outputs, states = tf.nn.dynamic_rnn(cells, X,
                                    initial_state=initial_state, dtype=tf.float32)


outputs = tf.reshape(outputs, [-1, hidden_size])
logits = layers.linear(outputs, vocab_size) # linear
#                                 ,activation_fn=None)

In [4]:
y_softmax = tf.nn.softmax(logits)
pred = tf.argmax(y_softmax, axis=1)
pred = tf.reshape(pred, [batch_size, -1])

In [78]:
# 시작 글자 생성
sentence = ["안녕", "나는", "너야"]
print(sentence)
print("Start with:", " ".join(sentence))

saver = tf.train.Saver()
sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
with tf.Session(config=sess_config) as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, "char-rnn_230000")
    
    # [배치사이즈, max_length, 100]
    vec = np.empty((1, len(sentence), 100)) # 시작 글자를 주고 다음 단어를 예측 - 시작으로 준 것을 전부 다 입력으로 넣겠다
    for i, word in enumerate(sentence):
        vec[:, i, :] = loader.song2vec.wv[word]
    
    # 매 이터레이션마다 글자 하나씩 생성
    state = sess.run(states, feed_dict={X: vec}) #입력단어 sentence 이후에 들어올 단어를 예측
    for i in range(15): # for문을 돌면서 풀어헤치는 중!!
        vec = loader.song2vec.wv[sentence[-1]].reshape(1, 1, 100)
        
        pred_char, state = sess.run([pred, states], 
            feed_dict={X: vec, initial_state: state}) # 원래 initial stete 는 0 이었으나
        # 이전 스텝에 갖고 있는 state 값을 다음 스텝에 넣어줌 // 입력단어들을 그 다음 스텝에 넣어주는 것임
        
        pred_char = loader.words[pred_char[0][-1]]
        sentence.append(pred_char)

for i, word in enumerate(sentence):
    print(word, end=" ")
    if (i+1) % 5 == 0:
        print()

['안녕', '나는', '너야']
Start with: 안녕 나는 너야
INFO:tensorflow:Restoring parameters from char-rnn_230000
안녕 나는 너야 contemplate 그림에 
read 예감이 웃는게_아니야 나없이 웃는게_아니야 
일_있네 기다렸지 되야 to 전부인 
다신 usual 욕먹는 